<a href="https://colab.research.google.com/github/silasabs/CoherentOptics/blob/main/examples/7.%20Clock%20Recovery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
if 'google.colab' in str(get_ipython()):    
    ! git clone -b main https://github.com/edsonportosilva/OptiCommPy
    from os import chdir as cd
    cd('/content/OptiCommPy/')
    ! pip install . 

# Clock Recovery

Os sistemas de comunicação digital processam amostras digitalizadas de formas de onda analógicas. Essas amostras devem ser coletadas em instantes específicos de tempo, e imperfeições nesse processo podem prejudicar significativamente a transmissão de dados. 

Erros de temporização podem ser classificados em erros de fase de amostragem e erros de frequência de amostragem. Erros de fase de amostragem aparecem quando a frequência do relógio do receptor é sincronizada com precisão com a frequência do relógio do transmissor, mas há um atraso constante com relação aos instantes de amostragem ideais. 

Em sistemas com erros de frequência de amostragem, os instantes de amostragem ideais variam ao longo do tempo, levando a uma resposta de frequência variável no tempo que não pode ser rastreada indefinidamente. Portanto, algoritmos de recuperação de clock, que corrigem adaptativamente erros de amostragem de frequência, são de fundamental importância.

## 7.1 Clock Recovery Architectures

Os esquemas de recuperação de relógio em sistemas digitais podem ser classificados de acordo com sua arquitetura em três categorias:

- **Esquemas analógicos:** Em um esquema puramente analógico, um circuito de entrada analógico extrai informações de tempo do sinal e atua diretamente. Tais estruturas de recuperação de relógio analógico não são usadas em receptores ópticos coerentes digitais, pois a detecção de erro de tempo pode ser implementada efetivamente no domínio digital.

<br>
<center>
    <img src="https://i.postimg.cc/4xHQqpC4/Screenshot-from-2024-10-04-18-21-31.png" width="900">
</center>
</br>

- **Esquemas Híbridos:** No esquema híbrido, o sinal recebido é amostrado e processado no domínio digital para extrair informações de tempo. Essas informações são então usadas para atuar no relógio de amostragem.

<br>
<center>
    <img src="https://i.postimg.cc/GmbsJYcS/Screenshot-from-2024-10-04-18-26-01.png" width="900">
</center>
</br>

- **Esquemas Digitais:** No esquema puramente digital, a extração do sinal de relógio, bem como os ajustes de tempo de amostragem são feitos de forma puramente digital.

<br>
<center>
    <img src="https://i.postimg.cc/N0SMVrct/Screenshot-from-2024-10-04-18-27-15.png" width="900">
</center>
</br>

Existem várias alternativas viáveis ​​para implementar a recuperação de relógio em sistemas ópticos coerentes digitais, incluindo esquemas híbridos analógico-digitais, esquemas puramente digitais ou uma combinação de ambos. Uma solução comum combina
um esquema híbrido analógico-digital para ajustes de tempo grosseiros e um esquema puramente digital para ajustes finos de tempo.

## Implementações nos domínios do tempo e frequência

Algoritmos de recuperação de clock podem ser implementados no domínio do tempo ou frequência. Algoritmos no domínio de frequência podem ser vantajosos do ponto de vista da complexidade se a cadeia de algoritmos DSP já implementa transformadas rápidas de Fourier (FFTs) e transformadas rápidas de Fourier inversas (IFFTs) para outras aplicações, como a compensação da dispersão cromática, evitando operações repetidas.

Um esquema híbrido típico de domínio de frequência implantado em sistemas de comunicações ópticas é mostrado abaixo. Um sinal de detector de erro de temporização (TED) é obtido no domínio de frequência após o sinal ser multiplicado pelo filtro de compensação CD, mas antes do IFFT.

<br>
<center>
    <img src="https://i.postimg.cc/DyTcHSv1/Screenshot-from-2024-10-04-18-44-55.png">
</center>
</br>

A saída do TED fornece um sinal de controle que, após a filtragem no filtro de loop (LF), é capaz de controlar o oscilador controlado por tensão (VCO) que atua nos ADCs. Um esquema puramente digital típico de domínio de tempo é mostrado abaixo.

O sinal de entrada $x(t)$ é primeiramente amostrado pelo ADC a uma taxa de amostragem fixa $1/Ta$ , onde $Ta$ é o período de amostragem. Assim, um sinal digitalizado de tempo discreto, com amostras $x[m] = x(mTa)$, é produzido na saída do ADC. O sinal digital é então encaminhado para um interpolador, que amostra o sinal nos tempos de amostragem desejados.

<br>
<center>
    <img src="https://i.postimg.cc/mkG7vnsT/Screenshot-from-2024-10-04-18-53-12.png">
</center>
</br>

A saída do interpolador gera amostras com taxa $1/T_I$ . Idealmente, como sistemas ópticos coerentes digitais trabalham com equalizadores $Ts/2$ espaçados fracionadamente, $T_I = Ts/2$. O caminho de feedback começa com um TED, fornecendo um sinal de controle que, após o LF, é capaz de controlar um oscilador controlado numericamente (NCO). A saída do NCO atua no interpolador, fechando o loop de fase digital (DPLL). Várias outras combinações também são possíveis, por exemplo, com recuperação de clock puramente digital no domínio da frequência.

## 7.2 Interpolador

Em esquemas de recuperação de clock puramente digitais, o propósito da interpolação é obter uma amostra da forma de onda do sinal em um determinado instante, com base em amostras coletadas em instantes vizinhos. 

<br>
<center>
    <img src="https://i.postimg.cc/XqptPxwN/Screenshot-from-2024-10-04-20-44-12.png">
</center>
</br>

Um conjunto de amostras de entrada espaçadas em $T_a$, indicadas na parte superior da figura, são usadas para gerar a amostra de saída no instante $nT_I$ . O processo é baseado na interpolação de uma amostra localizada em um intervalo fracionário $\mu_n T_a$ , tendo como ponto base uma amostra no instante $m_n T_a$ , onde $m_n = \frac{kT_I}{T_a}$. 
A interpolação já foi discutida no [Cap. 4](https://github.com/silasabs/CoherentOptics/blob/main/examples/4.%20Coherent%20Receiver%20Front-End.ipynb), no escopo da compensação de inclinação. A recuperação de clock usa algoritmos semelhantes, que são baseados em filtros lineares transversais. No entanto, diferentemente da compensação de inclinação, o esquema de recuperação de clock ajusta os coeficientes do filtro dinamicamente, símbolo por símbolo, com base na saída de um NCO. 


A interpolação pode ser implementada no domínio da frequência junto com a compensação de CD, ou usando um interpolador de domínio de tempo. Uma solução popular no domínio do tempo usa um interpolador cúbico de Lagrange. Os interpoladores cúbicos podem ter de 4 a 10 taps com banda passante crescente. Os coeficientes de um interpolador cúbico de 4 taps, para um instante de interpolação $t = m_nT_a + \mu_n T_a$ , são fornecidos por [2].

$$w_{n-2}\;=\;-\,{\frac{1}{6}}\mu_{n}^{3}+\,{\frac{1}{6}}\mu_{n};$$

$$w_{n-1}\;=\;\frac{1}{2}\mu_{n}^{3}+\frac{1}{2}\mu_{n}^{2}-\mu_{n};$$

$$w_{n}\;=\;-\,\frac{1}{2}\mu_{n}^{3}-\mu_{n}^{2}+\frac{1}{2}\mu_{n}+1;$$

$$w_{n+1}\;=\;{\frac{1}{6}}\mu_{n}^{3}+{\frac{1}{2}}\mu_{n}^{2}+{\frac{1}{3}}\mu_{n}.$$

# Referências

[1] Digital Coherent Optical Systems, Architecture and Algorithms

[2] L. Erup, F.M. Gardner, R.A. Harris, Interpolation in digital modems II. Implementation and performance. IEEE Trans. Commun. 41(6), 998–1008 (1993)